1. Install dependencies

In [1]:
!pip install torch torchvision transformers==4.36.0 datasets==3.6.0 evaluate accelerate timm

2. imports

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


3. Load the dataset

In [3]:
dataset = load_dataset("fcakyon/pokemon-classification", name="full", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/46.2M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4869 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1390 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/732 [00:00<?, ? examples/s]

4. Setup processor (preprocessing images)

In [4]:
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

5. Define transform

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

6. Dataset wrapper

In [6]:
class PokemonDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform, processor):
        self.dataset = dataset
        self.transform = transform
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        print(f"Accessing index: {idx}")
        print(f"Dataset type: {type(self.dataset)}")
        print(f"Dataset length: {len(self.dataset)}")
        item = self.dataset[idx]
        image = item["image"]   # raw PIL Image
        label = item["labels"]  # use actual label from dataset
        image = self.transform(image)
        return {"pixel_values": image, "labels": label}

7. Split into train/val

In [7]:
train_dataset = PokemonDataset(dataset["train"].select(range(1000)), transform, processor)
val_dataset   = PokemonDataset(dataset["train"].select(range(1000, 1200)), transform, processor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8)

8. Load pretrained ViT model

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=150,   # change according to your dataset
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([150]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([150, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

9. Training setup

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

10. Training loop

In [10]:
epochs = 2

for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Training loss: {avg_loss:.4f}")

Epoch 1


  0%|          | 0/125 [00:00<?, ?it/s]

Accessing index: 780
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 60
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 946
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 902
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 962
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 699
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 824
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 853
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


  1%|          | 1/125 [00:31<1:04:40, 31.29s/it]

Accessing index: 659
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 844
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 715
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 810
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 574
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 433
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 835
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 540
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


...

Accessing index: 441
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 102
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 967
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 549
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 617
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 127
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 861
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 14
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


 99%|█████████▉| 124/125 [28:49<00:14, 14.01s/it]

Accessing index: 361
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 796
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 228
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 234
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 972
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 637
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 582
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000
Accessing index: 749
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 1000


100%|██████████| 125/125 [29:02<00:00, 13.94s/it]

Training loss: 2.6468


11. Save model

In [11]:
model.save_pretrained("/content/drive/MyDrive/pokemon-vit")
processor.save_pretrained("/content/drive/MyDrive/pokemon-vit")

['/content/drive/MyDrive/pokemon-vit/preprocessor_config.json']

12. Load saved model and predict

In [12]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in val_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        preds = outputs.logits.argmax(-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct/total:.2%}")

Accessing index: 0
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 1
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 2
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 3
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 4
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 5
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 6
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 7
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 8
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 9
Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 200
Accessing index: 10
Dataset type: <class 'datasets.arrow_dat

Load pretrained

In [20]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch
from PIL import Image
from urllib.request import urlopen
from io import BytesIO

# load
model = AutoModelForImageClassification.from_pretrained("/content/drive/MyDrive/pokemon-vit")
processor = AutoImageProcessor.from_pretrained("/content/drive/MyDrive/pokemon-vit")

# move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# test on one image
url = "https://raw.githubusercontent.com/CesiumLabs/pokeres/refs/heads/main/img/4.png"
image = Image.open(BytesIO(urlopen(url).read())).convert("RGB")

inputs = processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1)

pred_id = probs.argmax(-1).item()
label = model.config.id2label[pred_id]

print(f"Predicted Pokémon: {label}")

Predicted Pokémon: LABEL_46


In [17]:
!ls drive/MyDrive/pokemon-vit/

config.json  model.safetensors	preprocessor_config.json
